# [Predictive Clinical Neuroscience Toolkit](https://github.com/amarquand/PCNtoolkit) 
# The Normative Modeling Framework for Computational Psychiatry Protocol
## Using Pre-trained Normative Models for Longitudinal Cortical Thickness Data Analysis

Created by [Barbora Rehák Bučková](https://twitter.com/BarboraRehak)

<div>
<img src="/home/barbora/Documents/Projects/PCNtoolkit-demo/tutorials/Long_NM/Protocol_overview.png" width="500">
</div>

# Data Preparation

## Install necessary libraries & grab data files

### Step 1.
Begin by cloning the GitHub repository using the following commands. This repository contains the necessary code and example data. Then install the python packages using pip and import them into the python environment (either Google Colab or using a local python installation on your computer).

In [2]:
! git clone https://github.com/predictive-clinical-neuroscience/PCNtoolkit-demo.git

In [ ]:
import os

# set this path to the git cloned PCNtoolkit-demo repository --> Uncomment whichever line you need for either running on your own computer or on Google Colab.
#wdir = '<path-to-your>/PCNtoolkit-demo' # if running on your own computer, use this line (change the path to match where you cloned the repository)
wdir ='/content/PCNtoolkit-demo' # if running on Google Colab, use this line

os.chdir(os.path.join(wdir,'tutorials','BLR_protocol'))